# The Basics with Agents SDK

We recently launched the [Agents SDK](https://openai.github.io/openai-agents-python/) to make it easier to orchestrate agentic workflows. In this cookbook, we have'll some fun composing a Golf Agent that can help with creating personalized practice plans and also make score predictions for scores around you.

- [Response API](https://platform.openai.com/docs/api-reference/responses)
- [File Search API](https://platform.openai.com/docs/guides/tools-file-search)
- [Web Search API](https://platform.openai.com/docs/guides/tools-web-search?api-mode=responses)

## Agent Workflow

For our design we divide the roles and resoponsibilities of agents by functionality. This architecture will help not only with dividing roles and responaibilities but also debugging and optimizations later on. Below are the following Agents and their scope 

![Local image](./Tee%20Time%20Finder.png)

- Triage Agent - responsible for understanding the user's initial intent and handing off to our other agents.
- Practice Planner Agent - responsibale for constructing a personalized and detailed practice plan. This agent will have access to the FileSearch API to query for relevant user shot informatin
- Score Predictor & Course Finder - responsible for finding nearby golf courses based on the user's request, making a score prediction for each fo the golf courses returned, and recommending club distance lengths for that day. This agent has access to the same FileSearch API to review shot data and also has access to the WebSearch API.


## Data Preparation & Vector Store Population

Before we can fire user requests into our Golf Agent, we need to upload the user's historical shot data into a Vector Store. Here is an example of the original golf data from a user's driving range practice session. 

| Date            | Player | Club Name | Club Type | Club Speed | Attack Angle | Club Path | Club Face | Face to Path | Ball Speed | Smash Factor | Launch Angle | Launch Direction | Backspin  | Sidespin  | Spin Rate | Spin Rate Type | Spin Axis | Apex Height | Carry Distance | Carry Deviation Angle | Carry Deviation Distance | Total Distance | Total Deviation Angle | Total Deviation Distance |
|-----------------|--------|-----------|-----------|-------------|---------------|------------|------------|----------------|--------------|----------------|----------------|------------------|------------|------------|-------------|----------------|------------|--------------|----------------|------------------------|--------------------------|----------------|------------------------|--------------------------|
| 1/4/25 17:01:39 | Brian  |           | 7 Iron    | 74.02      | 2.68         | 5.99      | 10.18     | 4.19           | 100.39      | 1.36           | 20.67         | 9.11              | 3501.39   | -366.16   | 3520.48    | Measured      | 5.97       | 22.77        | 143.31         | 11.92                  | 29.59                    | 156.29         | 12.01                  | 32.52                    |
| 1/4/25 17:02:08 | Brian  |           | 7 Iron    | 75.85      | -3.13        | 3.87      | 5.76      | 1.89           | 100.17      | 1.32           | 18.06         | 5.22              | 3549.35   | 61.95     | 3549.89    | Measured      | -1.0       | 19.23        | 140.03         | 5.18                   | 12.63                    | 154.17         | 5.11                   | 13.74                    |
| 1/4/25 17:02:52 | Jason  |           | 7 Iron    | 75.74      | -4.96        | 7.18      | 5.95      | -1.23          | 87.15       | 1.15           | 12.74         | 5.79              | 2423.86   | 1189.54   | 2700.02    | Estimated     | -26.14     | 6.79         | 91.81          | 0.92                   | 1.47                     | 122.10         | -0.35                  | -0.75                    |
| 1/4/25 17:03:49 | Herald |           | 7 Iron    | 75.99      | 4.53         | 5.91      | 5.20      | -0.71          | 96.95       | 1.28           | 14.49         | 5.01              | 4544.12   | -699.41   | 4597.63    | Measured      | 8.75       | 13.98        | 124.95         | 8.29                   | 18.01                    | 138.91         | 8.55                   | 20.66                    |
| 1/4/25 17:04:58 | Jason  |           | 7 Iron    | 77.08      | 7.36         | 3.77      | 3.15      | -0.62          | 102.56      | 1.33           |               | 3.06              | 1918.64   | 0.00      | 1918.64    | Measured      | 0.00       |              |                 |                         |                          |                |                        |                          |
| 1/4/25 17:05:42 | Brian  |           | 7 Iron    | 34.29      | 0.00         | 0.00      | 0.00      | 0.00           | 41.41       | 1.21           | 17.26         | -2.19             | 3052.55   | 0.00      | 3052.55    | Estimated     | 0.00       | 2.00         | 23.61          | -2.21                  | -0.91                    | 29.05          | -2.21                  | -1.12                    |
| 1/4/25 17:08:56 | Jason  |           | 7 Iron    | 75.77      | -4.19        | 6.90      | 10.15     | 3.25           | 98.76       | 1.30           | 19.33         | 9.20              | 5117.86   | 573.16    | 5149.85    | Measured      | -6.39      | 21.80        | 133.79         | 7.32                   | 17.04                    | 143.50         | 7.09                   | 17.72                    |
| 1/4/25 17:09:17 | Brian  |           | 7 Iron    | 76.39      | 2.79         | 3.26      | 6.68      | 3.42           | 104.35      | 1.37           | 15.02         | 5.89              | 3443.57   | -523.87   | 3483.19    | Measured      | 8.65       | 16.47        | 143.91         | 9.10                   | 22.75                    | 159.72         | 9.34                   | 25.93                    |
| 1/4/25 17:09:48 | Jason  |           | 7 Iron    | 76.35      | 0.35         | 2.09      | 6.56      | 4.47           | 102.63      | 1.34           | 14.20         | 5.65              | 5386.30   | -1082.16  | 5493.93    | Measured      | 11.36      | 16.61        | 135.01         | 10.57                  | 24.77                    | 146.59         | 10.88                  | 27.67                    |
| 1/4/25 17:10:26 | Lauren |           | 7 Iron    | 77.78      | 4.00         | 8.51      | 9.90      | 1.39           | 99.70       | 1.28           | 17.16         | 9.20              | 4462.13   | -68.54    | 4462.66    | Measured      | 0.88       | 18.77        | 135.86         | 10.13                  | 23.90                    | 147.86         | 10.09                  | 25.91                    |


In the perfect world, if we were a serious golfer, we could have added to the Note and Tag fields to writedown specific observations from each of these shots while practicing. Therefore, we're going to enrich this data by using a LLM to provide detailed shot feedback for each shot.

## Step 1: Data Preparation
In this step, we read the input CSV files, process them by combining header and unit information, and save the processed files for later use.

In [18]:
!pip install pandas openai tqdm

In [24]:
import os
import time
import random
import json
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

tqdm.pandas()  # Enable progress_apply for pandas operations

client = OpenAI()

SYSTEM_INSTRUCTIONS = """
You are a professional and very popular golf instructor with years of experience. You have a deep understanding of a variety of golf drills and exercises that can help golfers improve their game. You excel at synthesizing complex golf metrics from launch monitors and distilling them into comprehensible and actionable feedback and tips without being too technical. Convert it into lamens terms for high handicappers.
"""

def generate_feedback(row):
    prompt = f"""
    Review the following shot data and provide concise feedback:

    Date: {row.get('Date', 'N/A')}
    Club Type: {row.get('Club Type', 'N/A')}
    Club Speed: {row.get('Club Speed', 'N/A')} mph
    Attack Angle: {row.get('Attack Angle', 'N/A')} degrees
    Club Path: {row.get('Club Path', 'N/A')} degrees
    Club Face: {row.get('Club Face', 'N/A')} degrees
    Face To Path: {row.get('Face To Path', 'N/A')} degrees
    Ball Speed: {row.get('Ball Speed', 'N/A')} mph
    Smash Factor: {row.get('Smash Factor', 'N/A')}
    Launch Angle: {row.get('Launch Angle', 'N/A')} degrees
    Launch Direction: {row.get('Launch Direction', 'N/A')} degrees
    Backspin: {row.get('Backspin', 'N/A')} rpm
    Sidespin: {row.get('Sidespin', 'N/A')} rpm
    Spin Rate: {row.get('Spin Rate', 'N/A')} rpm
    Spin Rate Type: {row.get('Spin Rate Type', 'N/A')}
    Spin Axis: {row.get('Spin Axis', 'N/A')} degrees
    Apex Height: {row.get('Apex Height', 'N/A')} yards
    Carry Distance: {row.get('Carry Distance', 'N/A')} yards
    Carry Deviation Angle: {row.get('Carry Deviation Angle', 'N/A')} degrees
    Carry Deviation Distance: {row.get('Carry Deviation Distance', 'N/A')} yards
    Total Distance: {row.get('Total Distance', 'N/A')} yards
    Total Deviation Angle: {row.get('Total Deviation Angle', 'N/A')} degrees
    Total Deviation Distance: {row.get('Total Deviation Distance', 'N/A')} yards
    Note: {row.get('Note', 'N/A')}
    Tag: {row.get('Tag', 'N/A')}
    Air Density: {row.get('Air Density', 'N/A')} g/L
    Temperature: {row.get('Temperature', 'N/A')} °F
    Air Pressure: {row.get('Air Pressure', 'N/A')} kPa
    Relative Humidity: {row.get('Relative Humidity', 'N/A')}%

    Provide brief feedback and a short tag describing a drill that the golfer can do to improve their game such as "step through drill", "swing path drill", "slow motion swing", "shoulder turn check", "half swing pause", etc.
    Format your response as JSON:
    {{
        "feedback": "Your feedback here",
        "tag": "Your tag here"
    }}
    """
    response = client.responses.create(
        model="gpt-4o-mini",
        input=[
            {"role": "system", "content": SYSTEM_INSTRUCTIONS},
            {"role": "user", "content": prompt}
        ],
        text={
            "format": {
                "type": "json_object"
            }
        }
    )

    result = response.output_text
    return pd.Series(eval(result))

def process_and_enrich_csv(filepath, output_filepath):
    df = pd.read_csv(filepath)
    feedback_tags = df.progress_apply(generate_feedback, axis=1)
    df['Note'] = feedback_tags['feedback']
    df['Tag'] = feedback_tags['tag']
    df.to_csv(output_filepath, index=False)
    print(f"Updated CSV saved to {output_filepath}")

data_folder = os.path.join(os.path.dirname(os.getcwd()), "basics_of_agents_sdk/data")
raw_data_folder = os.path.join(data_folder, 'raw')
enriched_data_folder = os.path.join(data_folder, 'processed')

# Loop through each CSV file in the raw data folder with a progress bar
for filename in tqdm(os.listdir(raw_data_folder), desc="Processing CSV files"):
    if filename.endswith('.csv'):
        csv_path = os.path.join(raw_data_folder, filename)
        output_filepath = os.path.join(enriched_data_folder, filename)
        process_and_enrich_csv(csv_path, output_filepath)


Processing CSV files:   0%|          | 0/13 [00:55<?, ?it/s]


KeyboardInterrupt: 

## Step 2: Data Enrichment
In this step, we enrich the data by generating feedback and a tag for each row using a language model.

KeyboardInterrupt: 